In [13]:
import os
import sys
from pathlib import Path

project_root = Path('..').resolve()
sys.path.insert(0, str(project_root))

In [14]:
from src.core.extract import download_asset_data, preformat_data
from src.core.transform import add_all_indicators
from src.core.load import save_chart, save_table

from src.visualization.charts import FinancialChart

In [15]:
data = download_asset_data('AAPL')

data

[2025-11-19 10:19:52] [INFO] [src.core.extract] [>>] Começando donwload de dados do ativo AAPL.
C:\Users\GIULIANO\Documents\AAA Giu\projetos\finpipe\src\core\extract.py:55: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed
[2025-11-19 10:19:56] [INFO] [src.core.extract] [>>] Começando achatamneto de colunas.
[2025-11-19 10:19:56] [INFO] [src.core.extract] [<<] Achatamento de colunas concluido.
[2025-11-19 10:19:56] [INFO] [src.core.extract] [>>] Começando achatamneto de colunas.
[2025-11-19 10:19:56] [INFO] [src.core.extract] [<<] Achatamento de colunas concluido.
[2025-11-19 10:19:56] [INFO] [src.core.extract] [>>] Começando achatamneto de colunas.
[2025-11-19 10:19:56] [INFO] [src.core.extract] [<<] Achatamento de colunas concluido.
[2025-11-19 10:19:56] [INFO] [src.core.extract] [>>] Começando achatamneto de colunas.
[2025-11-19 10:19:56] [INFO] [src.core.extract] [<<] Achatamento de c

date,close_aapl,high_aapl,low_aapl,open_aapl,volume_aapl
datetime[ns],f64,f64,f64,f64,i64
2025-08-19 00:00:00,230.336716,232.644476,229.127896,231.05602,39402600
2025-08-20 00:00:00,225.791107,230.246794,225.551349,229.757264,42263900
2025-08-21 00:00:00,224.68219,226.300631,223.563279,226.050874,30621200
2025-08-22 00:00:00,227.539413,228.868127,225.191699,225.950957,42477800
2025-08-25 00:00:00,226.940002,229.077929,226.010895,226.260653,30983100
…,…,…,…,…,…
2025-11-12 00:00:00,273.470001,275.730011,271.700012,275.0,48398000
2025-11-13 00:00:00,272.950012,276.700012,272.089996,274.109985,49602800
2025-11-14 00:00:00,272.410004,275.959991,269.600006,271.049988,47431300


In [16]:
data = preformat_data(data)

data

[2025-11-19 10:19:56] [INFO] [src.core.extract] [>>] Começando pré-processamento de dados.
[2025-11-19 10:19:56] [INFO] [src.core.extract] [+] Mapping e rename de colunas concluído.
[2025-11-19 10:19:56] [INFO] [src.core.extract] [+] Removido qualquer linha com valor de fechamento (close) nulo. Convertido data para timezone de SP.
[2025-11-19 10:19:56] [INFO] [src.core.extract] [+] Adicionado coluna "symbol" e ordenado tabela de dados por data.
[2025-11-19 10:19:56] [INFO] [src.core.extract] [<<] Encerrando pré-processamento de dados.


date,symbol,open,high,low,close,volume
"datetime[ns, America/Sao_Paulo]",str,f64,f64,f64,f64,i64
2025-11-17 21:00:00 -03,"""aapl""",269.98999,270.709991,265.320007,267.440002,45634800
2025-11-16 21:00:00 -03,"""aapl""",268.820007,270.48999,265.730011,267.459991,45018300
2025-11-13 21:00:00 -03,"""aapl""",271.049988,275.959991,269.600006,272.410004,47431300
2025-11-12 21:00:00 -03,"""aapl""",274.109985,276.700012,272.089996,272.950012,49602800
2025-11-11 21:00:00 -03,"""aapl""",275.0,275.730011,271.700012,273.470001,48398000
…,…,…,…,…,…,…
2025-08-24 21:00:00 -03,"""aapl""",226.260653,229.077929,226.010895,226.940002,30983100
2025-08-21 21:00:00 -03,"""aapl""",225.950957,228.868127,225.191699,227.539413,42477800
2025-08-20 21:00:00 -03,"""aapl""",226.050874,226.300631,223.563279,224.68219,30621200


In [17]:
data = add_all_indicators(data)

data

date,symbol,open,high,low,close,volume,sma_20,sma_50,ema_12,ema_26,rsi_14,macd_12_26,macd_signal_9,macd_histogram,bb_middle_20,bb_upper_20,bb_lower_20,bb_std_20,atr_14,vwap
"datetime[ns, America/Sao_Paulo]",str,f64,f64,f64,f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2025-11-17 21:00:00 -03,"""aapl""",269.98999,270.709991,265.320007,267.440002,45634800,null,null,267.440002,267.440002,null,0.0,0.0,0.0,null,null,null,null,null,267.823334
2025-11-16 21:00:00 -03,"""aapl""",268.820007,270.48999,265.730011,267.459991,45018300,null,null,267.45083,267.450381,null,0.000448,0.000249,0.000199,null,null,null,null,null,267.858094
2025-11-13 21:00:00 -03,"""aapl""",271.049988,275.959991,269.600006,272.410004,47431300,null,null,269.386397,269.232325,null,0.154071,0.063291,0.09078,null,null,null,null,null,269.50638
2025-11-12 21:00:00 -03,"""aapl""",274.109985,276.700012,272.089996,272.950012,49602800,null,null,270.511291,270.271628,null,0.239662,0.123037,0.116625,null,null,null,null,null,270.671071
2025-11-11 21:00:00 -03,"""aapl""",275.0,275.730011,271.700012,273.470001,48398000,null,null,271.315162,271.013344,null,0.301818,0.17622,0.125597,null,null,null,null,null,271.278343
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2025-08-24 21:00:00 -03,"""aapl""",226.260653,229.077929,226.010895,226.940002,30983100,235.55865,248.83298,232.955302,238.445875,38.422328,-5.490572,-5.325038,-0.165534,235.55865,249.264511,221.85279,6.85293,5.979204,252.420192
2025-08-21 21:00:00 -03,"""aapl""",225.950957,228.868127,225.191699,227.539413,42477800,234.144022,248.00798,232.122062,237.63109,41.94534,-5.509028,-5.361836,-0.147192,234.144022,244.460763,223.82728,5.158371,5.693767,252.081776
2025-08-20 21:00:00 -03,"""aapl""",226.050874,226.300631,223.563279,224.68219,30621200,233.115019,247.099461,230.977436,236.664333,43.248221,-5.686897,-5.426848,-0.260049,233.115019,242.851565,223.378473,4.868273,5.448288,251.820876


In [18]:
chart_dark = FinancialChart(data, {'theme':'dark'})
figure_dark = chart_dark.create_comprehensive_chart()
save_chart(figure=figure_dark, path=os.path.join(project_root, r'output\charts'), chart_name='test_darktheme', format='png')

figure_dark.show()

[2025-11-19 10:19:56] [INFO] [src.core.load] [>>] Salvando chart.
[2025-11-19 10:20:06] [INFO] [src.core.load] [+] Chart salvo em "C:\Users\GIULIANO\Documents\AAA Giu\projetos\finpipe\output\charts
[2025-11-19 10:20:06] [INFO] [src.core.load] [<<] Encerrando processo.


In [19]:
chart_light = FinancialChart(data, {'theme':'light'})
figure_light = chart_light.create_comprehensive_chart()
save_chart(figure=figure_light, path=os.path.join(project_root, r'output\charts'), chart_name='test_lighttheme', format='jpg')

figure_light.show()

[2025-11-19 10:20:06] [INFO] [src.core.load] [>>] Salvando chart.
[2025-11-19 10:20:14] [INFO] [src.core.load] [+] Chart salvo em "C:\Users\GIULIANO\Documents\AAA Giu\projetos\finpipe\output\charts
[2025-11-19 10:20:14] [INFO] [src.core.load] [<<] Encerrando processo.


In [20]:
chart = FinancialChart(data)
chart.set_theme('dark')
figure_dynamic = chart.create_simple_price_chart()
save_chart(figure=figure_dynamic, path=os.path.join(project_root, r'output\charts'), chart_name='simple_chart_darktheme', format='jpg')

figure_dynamic.show()

[2025-11-19 10:20:14] [INFO] [src.core.load] [>>] Salvando chart.
[2025-11-19 10:20:24] [INFO] [src.core.load] [+] Chart salvo em "C:\Users\GIULIANO\Documents\AAA Giu\projetos\finpipe\output\charts
[2025-11-19 10:20:24] [INFO] [src.core.load] [<<] Encerrando processo.


In [21]:
chart.set_theme('light')
figure_dynamic = chart.create_simple_price_chart()
save_chart(figure=figure_dynamic, path=os.path.join(project_root, r'output\charts'), chart_name='simple_chart_lighttheme', format='png')

figure_dynamic.show()

[2025-11-19 10:20:24] [INFO] [src.core.load] [>>] Salvando chart.
[2025-11-19 10:20:33] [INFO] [src.core.load] [+] Chart salvo em "C:\Users\GIULIANO\Documents\AAA Giu\projetos\finpipe\output\charts
[2025-11-19 10:20:33] [INFO] [src.core.load] [<<] Encerrando processo.
